In [ ]:
%load_ext autoreload
%autoreload 2
import transformers
import torch
import os.path
from os.path import join
import math
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from mcllm.model.llm import *
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import torch.utils.data as data
from pprint import pprint

import lightning.pytorch as pl
import mcllm.config
import mcllm.model
from pandas.api.types import is_numeric_dtype

In [ ]:
openml_dir = join(mcllm.config.path_to_repo, 'data/openml')
csvs = [f for f in os.listdir(openml_dir) if f.endswith('csv')]
print('n_dsets', len(csvs))

In [28]:
# process csvs to make them smaller
for i, csv in enumerate(csvs):
    df = pd.read_csv(join(openml_dir, csv))
    assert 'target' in df.columns
    for col in df.columns:
        assert is_numeric_dtype(df[col])
    assert not df.isnull().values.any()

    # randomly subsample first 100 columns
    N = 3000
    print(i, csv, df.shape)
    if len(df.columns) > N:
        target = df[df.columns[-1]]
        df = df[df.columns[:-1]].sample(N, axis=1, random_state=42)
        df['target'] = target
        # df = df.join(target, axis=1)
    if len(df) > N:
        df = df.sample(N, axis=0, random_state=42)
    df.to_csv(join(openml_dir, csv))

0 FOREX_audsgd_hour_Close.csv (43825, 11)
1 FOREX_eurrub_hour_High.csv (43825, 11)
2 pendigits.csv (10992, 17)
3 phoneme_seed_1_nrows_2000_nclasses_10_ncols_100_stratify_True.csv (2000, 6)
4 Satellite_seed_3_nrows_2000_nclasses_10_ncols_100_stratify_True.csv (2000, 37)
5 MiniBooNE_seed_1_nrows_2000_nclasses_10_ncols_100_stratify_True.csv (2000, 51)
6 CreditCardSubset.csv (14240, 31)
7 FOREX_eurusd_day_High.csv (1837, 11)
8 robert_seed_3_nrows_2000_nclasses_10_ncols_100_stratify_True.csv (2000, 101)
9 fri_c4_1000_10.csv (1000, 11)
10 1StudentPerfromance.csv (1000, 4)
11 shuttle_seed_0_nrows_2000_nclasses_10_ncols_100_stratify_True.csv (2000, 10)
12 FOREX_eursek_hour_High.csv (43825, 11)
13 wilt_seed_3_nrows_2000_nclasses_10_ncols_100_stratify_True.csv (2000, 6)
14 FOREX_usddkk_minute_Close.csv (375840, 11)
15 BNG_pendigits_.csv (1000000, 17)
16 FOREX_eurnzd_hour_Close.csv (43825, 11)
17 fabert_seed_0_nrows_2000_nclasses_10_ncols_100_stratify_True.csv (2000, 101)
18 poker_hand.csv (10250